# extract text and its position

In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO


def convert_pdf_to_txt(path1, path2, num):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path1, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for i, page in enumerate(PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True)):
        interpreter.process_page(page)
        if i >= num:
            break
    text = retstr.getvalue()
    file_name = 'page' + str(num)
    f = open(path2+file_name, 'w')
    f.write(text)
    f.close()
    #text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()



In [2]:
path1 = '/home/zhongzhu/Desktop/Project2/Test2/CORE.pdf'
path2 = '/home/zhongzhu/Desktop/Project2/Test2/temp/'
convert_pdf_to_txt(path1, path2, 292)

# record positions

In [3]:
import re

path1 = '/home/zhongzhu/Desktop/Project2/Test2/CORE.pdf'
path2 = '/home/zhongzhu/Desktop/Project2/Test2/temp/'

def record_pos(path, page, LINE_HEIGHT = 1):
    f = open(path2+'page'+str(page), 'r')
    boxes = f.read().split('TEXT_BOX')
    f.close()

    contents = {}
    for content in boxes:
        c = content.split(';')
        if(c[0].find('left:') > 0): #if not empty
            l = c[0][c[0].find('left:')+5:c[0].find('px')]
            t = c[1][c[1].find('top:')+4:c[1].find('px')]
            w = c[2][c[2].find('width:')+6:c[2].find('px')]
            h = c[3][c[3].find('height:')+7:c[3].find('px')]

            text = "".join(c[4:])
            lines = text.split('\n')
                
            tempCount = 0
            for line in lines:
                if re.search(r'[A-Za-z0-9,.?!]', line):
                    if not line.decode('utf-8') in contents:
                        contents[line.decode('utf-8')] = [(int(l), int(t)+tempCount, int(w), int(h))]
                    else:
                        contents[line.decode('utf-8')].append((int(l), int(t)+tempCount, int(w), int(h)))
                tempCount += LINE_HEIGHT ### default value 1
    return contents


In [4]:
contents = record_pos(path2, 292)
for pos in contents:
    print(pos)
    print(contents[pos])

installed storm drain from manhole #34 to existing drain on 
[(307, 22128, 162, 211)]
ISS- insulate pipes in area C
[(307, 74510, 174, 371)]
 install hardware on interior gym doors
[(307, 177029, 157, 22)]
complete south exterior wall
[(307, 124228, 166, 76)]
remainder of north parking lot
[(307, 175479, 167, 209), (307, 175482, 167, 209)]
Synergy - pulling wire to roof AHU's
[(307, 174659, 173, 86), (307, 178180, 175, 286)]
 ADS
[(201, 199205, 32, 22), (201, 199245, 32, 22), (201, 203210, 32, 22), (201, 203258, 32, 22), (201, 203298, 32, 22), (201, 203338, 32, 22), (201, 203386, 20, 22), (201, 208717, 11, 9), (201, 209019, 20, 22), (201, 209067, 11, 9), (201, 209115, 11, 9), (201, 209163, 11, 9), (201, 209203, 11, 9), (201, 214555, 20, 22), (201, 214595, 20, 22), (201, 214635, 11, 9), (201, 214958, 20, 22), (201, 214998, 20, 22), (201, 221140, 20, 22), (201, 221180, 20, 22), (201, 221220, 20, 22), (201, 221259, 33, 22), (201, 221299, 33, 62)]
 installed FDC connections for standpipes 

In [5]:
filted_contents = {}
for text in contents:
    if len(contents[text]) > 200:  ###set at beginning#####################
        filted_contents[text] = contents[text]
        print(text + ' ' +str(len(filted_contents[text])))

Quality Iron Fabricators Inc. 269
Date: 292
 Project: 292
Allied Construction, LLC 222
  4 229
OS 345
  3 299
  2 405
  1 247
  5 235
  8 367
Location: 292
 Bernhard Mechanical Contractors, L.L.C. 316
  16 214
  10 238
F 1459
Is Today Claimed as Weather Day? 292
CORE Construction Services, LLC 229
 OS 368
Bernhard Mechanical Contractors, L.L.C. 636
 Tanner Broughton 218
4801 Maid Marion Drive 292
 Sherwood Forest Elementary School 292
N 286
 Fair Working Days To Date: 292
Day: 292
Report #: 292
Critical Path Work Effected? 292
Cordeiro LLC 288
 Low 292
Phoenix Constructors LLC 314
 Weather Condition: 291
 F 1237
 2 360
 3 275
 1 544
 6 202
 4 224
 5 203
 N 264
Gulf South Engineering &amp Testing Inc 254
Roofing Solutions LLC 393
Temp: 292
Malone All Star JV, LLC 506
 8 448
Precipitation Amount:  273
Weather Delays To Date: 292
 14-02-002 292
 Ellen Heck 213
 Job: 292
 Marybeth Leonard 211
CFI Insulation Inc 248
High 292
 10 222


In [13]:
convert_pdf_to_txt(path1, path2, 5)

# get the positions of the first page
first_page_contents = record_pos(path2, 5)

# search neighbors

In [14]:
from itertools import product
neighbors = {}
    
for line1, line2 in product(filted_contents, first_page_contents):
    if(line1 != line2 and line1 in first_page_contents
       and abs(int(first_page_contents[line1][0][0])-int(first_page_contents[line2][0][0]))<1000    ###set at beginning
       and abs(int(first_page_contents[line1][0][1])-int(first_page_contents[line2][0][1]))<30): ###set at beginning
        key = line1
        value = line2
        if not key in neighbors:
            neighbors[key] = []
        neighbors[key].append(value)

# GUI

In [15]:
import Tkinter as tk
import ttk

win = tk.Tk()
win.title("Set Pairs")   

ttk.Label(win, text="Select category:").grid(column=0, row=0)   
ttk.Label(win, text="Bind content").grid(column=2, row=0)      


def select():
    dropdown2['values'] = neighbors[category.get()]  
    dropdown2.current(0)    
    
# selecte button
action = ttk.Button(win, text="Select", command=select)
action.grid(column=1, row=1)   

pairs = {}
# 
def bind():   
    #key = contents[category.get()]
    #new_value = contents[content.get()]
    
    key = category.get()
    a = contents[category.get()][0]
    b = contents[content.get()][0]
    new_value = (b[0]-a[0], b[1]-a[1], b[2]-a[2], b[3]-a[3])
    
    if not key in pairs:
        pairs[key] = set([])
    pairs[key].add(new_value)
    dropdown2.current(0) 
    

# bind button
action = ttk.Button(win, text="Bind", command=bind)
action.grid(column=3, row=1) 

# creat a drop-down list
category = tk.StringVar()
dropdown1 = ttk.Combobox(win, width=60, height=20, textvariable=category)
dropdown1['values'] = list(neighbors.keys())     
dropdown1.grid(column=0, row=1)      
#dropdown1.current(0) 

# create another drop-down list
content = tk.StringVar()
dropdown2 = ttk.Combobox(win, width=60, textvariable=content)    
dropdown2.grid(column=2, row=1)        

win.mainloop()      # window

In [16]:
for i in pairs:
    print i
    print pairs[i]

Day:
set([(59, 0, 6, 0)])
Is Today Claimed as Weather Day?
set([(169, 21, -100, -24)])
Date:
set([(59, 0, 6, 0)])
 Job:
set([(59, 0, 6, 0)])
 Project:
set([(47, 0, 82, -2)])
Critical Path Work Effected?
set([(169, 20, -100, -24)])


In [17]:
def find_content(content_map, content_pos):
    
    content = []
    for c in content_map:
        m = 1 #### maximal error allowed#########################
        for pos in content_map[c]:
            error = abs(content_pos[0]-pos[0]) + abs(content_pos[1]-pos[1])
            #print error
            if error < m:
                content.append(c)
    #print(m)
    return content
    
data = {}
content_map = record_pos(path2, 500)
for cat in pairs:
    #if cat in content_map:  #unnecessary??
    for pos in content_map[cat]:
        content = []
        for shift in pairs[cat]:
            content_pos = (pos[0]+shift[0], pos[1]+shift[1], pos[2]+shift[2], pos[3]+shift[3])
            content.append(find_content(content_map, content_pos))
        if cat in data: #unnecessary??
            data[cat].append(content)
        else:
            data[cat] = [content]
           


In [18]:
for i in data:
    print i
    for j in data[i]:
        print j


Day:
[[u'Saturday']]
[[u'Sunday']]
[[u'Tuesday']]
[[u'Thursday']]
[[u'Monday']]
[[u'Monday']]
[[u'Monday']]
[[u'Monday']]
[[u'Monday']]
[[]]
[[u'Monday']]
[[u'Monday']]
[[u'Friday']]
[[u'Monday']]
[[u'Saturday']]
[[u'Monday']]
[[]]
[[u'Monday']]
[[u'Monday']]
[[u'Monday']]
[[u'Saturday']]
[[u'Monday']]
[[u'Sunday']]
[[u'Sunday']]
[[]]
[[u'Saturday']]
[[u'Saturday']]
[[u'Monday']]
[[u'Monday']]
[[u'Monday']]
[[u'Monday']]
[[u'Tuesday']]
[[]]
[[u'Thursday']]
[[u'Monday']]
[[u'Tuesday']]
[[u'Monday']]
[[u'Monday']]
[[u'Thursday']]
[[u'Saturday']]
[[u'Sunday']]
[[]]
[[u'Sunday']]
[[u'Tuesday']]
[[u'Sunday']]
[[u'Friday']]
[[u'Sunday']]
[[u'Friday']]
[[u'Monday']]
[[u'Tuesday']]
[[u'Monday']]
[[u'Monday']]
[[u'Monday']]
[[u'Thursday']]
[[u'Monday']]
[[u'Friday']]
[[]]
[[u'Thursday']]
[[u'Monday']]
[[u'Monday']]
[[u'Monday']]
[[u'Thursday']]
[[u'Monday']]
[[u'Friday']]
[[u'Sunday']]
[[u'Thursday']]
[[u'Monday']]
[[u'Tuesday']]
[[u'Monday']]
[[u'Friday']]
[[u'Friday']]
[[u'Monday']]
[[]]
[[u'

# generate CSV table

In [19]:
import csv
f = open('generated_table_core.csv', 'w')
writer = csv.writer(f)

for i, cat in enumerate(data):
        writer.writerow([cat] + data[cat])
f.close()

# test

In [1]:
for i in content_map:
    print i
    print content_map[i]

NameError: name 'content_map' is not defined

In [94]:
len(data[' WEATHER'])

1410

In [62]:
contents = record_pos(path2, 14)
for pos in contents:
    print(pos)
    print(contents[pos])

 Employer * * * * * *
(95, 10799, 71, 23)
None today * * * * * * * * * * * * * * * *
(95, 5474, 175, 18)
 PREP/INITIAL DATES * * * * *
(75, 8658, 75, 10)
 The following activity was started today:
(85, 10613, 156, 10)
 No contractors had their first or last day on site today * * * * * * *
(85, 10755, 206, 10)
 WEATHER * * * * * *
(319, 9436, 229, 28)
 QUALITY CONTROL INSPECTIONS AND RESULTS * * *
(95, 7013, 168, 18)
 DACA67-02-D-2012  0004 * * * * * * * * * * * * *
(424, 11105, 89, 10)
DATE * * * * *
(417, 4324, 16, 22)
None today * * * * * * * * * * * * *
(95, 5330, 168, 18)
Worked *
(508, 2721, 26, 20)
0.0 * * * *
(441, 2722, 36, 44)
 Evergreene Construction, L.C. * * * * * * *
(82, 11970, 102, 10)
 LABOR HOURS * * * * * *
(75, 9859, 53, 10)
 LABOR HOURS
(75, 614, 53, 10)
 No accidents reported today * *
(85, 4445, 109, 10)
 CONTRACTORS ON SITE (Report first and/or last day contractors w ere on site) * * * * * *
(75, 10741, 278, 10)
 General Comments:
(85, 267, 78, 10)
 QA/QC PUNCH L